In [ ]:
import tensorflow as tf

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

In [ ]:
layer_names = [
            'block_1_expand_relu',  # 64x64
            'block_3_expand_relu',  # 32x32
            'block_6_expand_relu',  # 16x16
            'block_13_expand_relu',  # 8x8
            'block_16_project',  # 4x4
        ]
layers = [base_model.get_layer(name).output for name in layer_names]

In [ ]:
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

In [ ]:
model_img_file = 'model.png'
tf.keras.utils.plot_model(down_stack, to_file=model_img_file, 
                          show_shapes=False, 
                          show_layer_activations=False, 
                          show_dtype=False,
                          show_layer_names=True )

In [ ]:
import glob
import numpy as np
import tensorflow as tf
import SimpleITK as sitk


class Pre_process:
    """Data Loader class"""

    def pad_images(self, img=None, target_width: int = 64, target_height: int = 64):
        """Padding the sizes fo the images
        Args:
            img (np.array): input image to be padded
            target_width (int): width of the frame
            target_height (int): height of the frame
        Return:
            padded_img (np.array): Padded image
        """
        padded_image = np.pad(img, [(0, target_width - img.shape[0]),
                                    (0, target_height - img.shape[1])], mode='constant')

        return padded_image

    def normalize_images(self, img=None, normalization_type='divide_255'):
        """
        Args:
            img: numpy 4D array
            normalization_type: `str`, available choices:
                - divide_255
                - divide_256
                - by_chanels
        """
        if normalization_type == 'divide_255':
            images = img / 255
        elif normalization_type == 'divide_256':
            images = img / 256
        elif normalization_type is None:
            pass
        else:
            raise Exception("Unknown type of normalization")
        return img

In [ ]:
class Chunk_data_loader(tf.keras.utils.Sequence):
    """ Loads data in chunks and useful when the process are getting out of memory
    Args:
        batch_size (int): number of samples in one batch
        input_img_paths (str): image paths
        label_img_paths (str): label paths
    Returns:
        image_batch, label_batch
    """

    def __init__(self, batch_size, input_img_paths, label_img_paths):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.label_img_paths = label_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""

        prepro = Pre_process()
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i: i + self.batch_size]
        batch_target_img_paths = self.label_img_paths[i: i + self.batch_size]

        # Loading images
        img_batch_array = []
        #print(batch_input_img_paths)
        for file_name in batch_input_img_paths:
            img = sitk.ReadImage(file_name)
            img = sitk.GetArrayFromImage(img)

            img_array = []
            for frame in img:
                # Padding the image and normalizing it
                temp = prepro.pad_images(frame, target_width=64, target_height=64)
                temp = prepro.normalize_images(temp)
                img_array.append(temp)
            img_batch_array.append(img_array)
        img_batch_array = np.array(img_batch_array)

        # Loading label
        label_batch_array = []
        for file_name in batch_target_img_paths:
            img = sitk.ReadImage(file_name)
            img = sitk.GetArrayFromImage(img)

            img_array = []
            for frame in img:
                # Padding the image and converting it into int
                temp = prepro.pad_images(frame, target_width=64, target_height=64)
                temp = temp.astype(int)
                img_array.append(temp)
            label_batch_array.append(img_array)
        label_batch_array = np.array(label_batch_array)

        return img_batch_array, label_batch_array

In [ ]:
image_path = "D:/mine/AISummer/project2/data/Task04_Hippocampus/imagesTr/"
label_path = "D:/mine/AISummer/project2/data/Task04_Hippocampus/labelsTr/"
batch_size = 8


In [ ]:
import random
val_samples = 20

def data_generator():
    input_img_paths = []
    full_path = image_path + "*.*"
    for file_name in glob.iglob(full_path):
        input_img_paths.append(file_name)
    input_img_paths = np.array(input_img_paths)


    input_label_paths = []
    full_path = label_path + "*.*"
    for file_name in glob.iglob(full_path):
        input_label_paths.append(file_name)
    input_label_paths = np.array(input_label_paths)

    random.Random(1337).shuffle(input_img_paths)
    train_input_img_paths = input_img_paths[:-val_samples]
    train_input_label_paths = input_label_paths[:-val_samples]

    data_gen = Chunk_data_loader(batch_size, train_input_img_paths, train_input_label_paths)
    
    x, y = data_gen.__getitem__(2)
    
    print(x.shape, y.shape)
    
    return data_gen

In [ ]:
train_gen = data_generator()

In [ ]:
train_gen

In [ ]:
x , y = train_gen.next() 

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dropout, Input, MaxPool2D, Conv2DTranspose, concatenate

layer_filter = [16, 32, 64, 128, 256]

In [2]:
inputs = tf.keras.layers.Input(shape=[64, 64, 1])
x = inputs

In [3]:
class CNNBlocks:
    def __init__(self,  kernel_size):
        self.activation = "relu"
        self.reg = tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)
        self.kernel = kernel_size
        self.dropout = 0.1
        
    def conv_down(self, n_conv, inputs):
        
        cd = Conv2D(n_conv, self.kernel, activation=self.activation,
                    kernel_regularizer=self.reg, padding='same')(inputs)
        cd = Dropout(self.dropout)(cd)
        cd = Conv2D(n_conv, self.kernel, activation=self.activation,
                    kernel_regularizer=self.reg, padding='same')(cd)
        
        return cd
        
        
    def concat(self, n_conv, inputs, skip):
        con = Conv2DTranspose(n_conv, (2, 2), strides=(2, 2), padding='same')(inputs)
        con = concatenate([con, skip])
        
        return con

In [33]:
conv_block = CNNBlocks(kernel_size=3)

d1 = conv_block.conv_down(layer_filter[0], inputs)
p1 = MaxPool2D((2, 2))(d1)

d2 = conv_block.conv_down(layer_filter[1], p1)
p2 = MaxPool2D((2, 2))(d2)

d3 = conv_block.conv_down(layer_filter[2], p2)
p3 = MaxPool2D((2, 2))(d3)

d4 = conv_block.conv_down(layer_filter[3], p3)
p4 = MaxPool2D((2, 2))(d4)

d5 = conv_block.conv_down(layer_filter[4], p4)


u6 = conv_block.concat(layer_filter[3], d5, d4)
u6 = conv_block.conv_down(layer_filter[3], u6)

u7 = conv_block.concat(layer_filter[2], u6, d3)
u7 = conv_block.conv_down(layer_filter[2], u7)

u8 = conv_block.concat(layer_filter[1], u7, d2)
u8 = conv_block.conv_up(layer_filter[1], u8)

u9 = conv_block.concat(layer_filter[0], u8, d1)
u9 = conv_block.conv_up(layer_filter[0], u9)

outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(u9)

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

In [34]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 conv2d_235 (Conv2D)            (None, 64, 64, 16)   160         ['input_1[0][0]']                
                                                                                                  
 dropout_111 (Dropout)          (None, 64, 64, 16)   0           ['conv2d_235[0][0]']             
                                                                                                  
 conv2d_236 (Conv2D)            (None, 64, 64, 16)   2320        ['dropout_111[0][0]']            
                                                                                           

 conv2d_transpose_36 (Conv2DTra  (None, 64, 64, 16)  2064        ['conv2d_250[0][0]']             
 nspose)                                                                                          
                                                                                                  
 concatenate_35 (Concatenate)   (None, 64, 64, 32)   0           ['conv2d_transpose_36[0][0]',    
                                                                  'conv2d_236[0][0]']             
                                                                                                  
 conv2d_251 (Conv2D)            (None, 64, 64, 16)   4624        ['concatenate_35[0][0]']         
                                                                                                  
 dropout_119 (Dropout)          (None, 64, 64, 16)   0           ['conv2d_251[0][0]']             
                                                                                                  
 conv2d_25

In [4]:
import numpy as np

x = np.zeros((31, 64, 64))
y = np.ones((5, 64, 64))

a = np.vstack((x, y))
a.shape

(36, 64, 64)